In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-128270")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-128270
Azure region: southcentralus
Subscription id: a7e82b92-bbed-4497-a8df-009b71b94f9c
Resource group: aml-quickstarts-128270


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# choose a name for your cluster
# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

cluster_name = "spirostestest"

# Choose a name for your CPU cluster
compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
# create the cluster
compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os

# Specify parameter sampler
### YOUR CODE HERE ###
ps = RandomParameterSampling(
    {
        '--C':uniform(0.001,100),
        '--max_iter':(1000)
})

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")
    
# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory="./training/", entry_script="train.py",compute_target=compute_target)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,
                                        hyperparameter_sampling=ps,
                                        policy=policy,
                                        primary_metric_name='Accuracy',
                                        primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                        max_total_runs=20)

In [5]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

best_hd_run = exp.submit(config=hyperdrive_config)
RunDetails(best_hd_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [6]:
import joblib
# Get your best run and save the model from that run.

best_run = best_hd_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

joblib.dump(best_run.id, 'log_reg.joblib')


print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])
print('\n C:',parameter_values[1])
print('\n Max Itter:',parameter_values[3])

Best Run Id:  HD_64ecc47b-2f6e-4058-87ab-d8411e88a61b_4

 Accuracy: 0.912797167425392

 C: 39.77811146745635

 Max Itter: 1000


In [10]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
ds = TabularDatasetFactory.from_delimited_files("https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")


In [24]:
import sys  
sys.path.insert(0, 'training')
import train
import pandas as pd

# Use the clean_data function to clean your data.
x, y = train.clean_data(ds)
final_dataset = pd.concat([x,y], axis=1)

In [30]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=final_dataset,
    label_column_name="y",
    n_cross_validations=5)

In [ ]:
# Submit your automl run

from azureml.core.experiment import Experiment

auto_ml_exp = Experiment(ws, "spiros_automl_exp")
run = auto_ml_exp.submit(config=automl_config, show_output=True)

Running on local machine
Parent Run ID: AutoML_5241fca2-5508-4f13-abd7-3b45ee32c7bb

Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely p

In [1]:
# Retrieve and save your best automl model.

automl_best_run, automl_fitted_model = run.get_output()
print(automl_fitted_model)

# Save Model

joblib.dump(automl_fitted_model("best_automl_model.joblib")


SyntaxError: unexpected EOF while parsing (<ipython-input-1-e87bca887375>, line 8)

In [ ]:
# Resource cleaning
compute_target.delete()